In [1]:
import json
import re
from typing import Literal, Optional
from pydantic import BaseModel

from langchain_ollama import ChatOllama
import os
import requests

MODEL_API = os.getenv("MODEL_API", "http://localhost:11434")
# EMBED_MODEL = os.getenv("EMBED_MODEL", "qwen3-embedding:0.6b")
# LLM_MODEL=os.getenv("LLM_MODEL","qwen2.5:3b-instruct-q8_0")
# LLM_MODEL=os.getenv("LLM_MODEL1","gemma3:1b")
# LLM_MODEL=os.getenv("LLM_MODEL1","qwen2.5:1.5b-instruct")
# LLM_MODEL=os.getenv("LLM_MODEL1","qwen2.5:1.5b")
# LLM_MODEL=os.getenv("LLM_MODEL1","qwen2.5:0.5b")   
LLM_MODEL=None
# OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://ollama:11434") 


QueryComplexity = Literal["SIMPLE", "COMPLEX"]
domainType = Literal["LAW", "TECHNICAL", "FINANCE", "MEDICAL","MATHEMATICS", "GENERAL", "OTHER"]


class RouterOutput(BaseModel):
    complexity: QueryComplexity
    confidence: float
    reason: str
    domain: domainType


def safe_json_parse(text: str):
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if match:
            return json.loads(match.group())
        raise ValueError("Invalid JSON from LLM")


def route_query(query: str) -> RouterOutput:
    prompt = f"""
    You are a routing classifier for a RAG system.

    Your task is to classify a user query by:
    1) Complexity
    2) Domain (knowledge area)

    RULES:
    - If the reason implies basic factual knowledge → complexity MUST be SIMPLE
    - If the reason implies analysis or deep reasoning → complexity MUST be COMPLEX
    - Domain MUST match the reason
    - NEVER contradict your own reason

    Confidence score:
    - this is a number between 0 and 1 decimal number representing how confident you are in your classifcation.
    - any complexity you classify always tell me how confident on that classification.

    Domain:
    - LAW
    - TECHNICAL
    - FINANCE
    - MEDICAL
    - MATHEMATICS
    - GENERAL
    - OTHER

    Return ONLY valid JSON in this format:
    {{
    "complexity": "SIMPLE" | "COMPLEX",
    "confidence": number,
    "domain": "LAW" | "TECHNICAL" | "FINANCE" | "MEDICAL" | "MATHEMATICS" | "GENERAL" | "OTHER",
    "reason": "brief justification"
    }}
    anylize the reason you give and find complexity and confidence based on the query justification.

    User query:
    "{query}"
    """


    try:
        data = {
            "model": LLM_MODEL,
            "prompt": prompt,
            "format": "json",
            "options": {
                "temperature": 0,
                "top_p": 0.9,
                "repeat_penalty": 1.1,
                "num_predict": 256
            }
        }

        response = requests.post(f"{MODEL_API}/api/generate", json=data, timeout=30)

        # --- Collect streamed chunks ---
        fragments = []
        for line in response.text.strip().splitlines():
            obj = json.loads(line)
            if "response" in obj:
                fragments.append(obj["response"])

        # --- Join all fragments into one JSON string ---
        full_text = "".join(fragments)

        # --- Parse the complete JSON ---
        parsed = json.loads(full_text)

        # return RouterOutput(**parsed)
        return parsed
    except Exception as e:
        print("[Router Error]", e)
        return RouterOutput(
            complexity="SIMPLE",
            confidence=0.5,
            reason="Fallback due to routing error"
        )



import pandas as pd


QUERIES = {
    "What is the capital of France?": "SIMPLE",
    "Explain ship classification and statutory regulation": "COMPLEX",
    "How does compound interest work?": "SIMPLE",
    "What are the symptoms of diabetes?": "SIMPLE",
    "Solve the equation x^2 - 4x + 4 = 0": "SIMPLE",
    "Describe the process of photosynthesis.": "SIMPLE",
    "What are the main causes of climate change?": "COMPLEX",
    "Explain the theory of relativity.": "COMPLEX",
    "How do vaccines work?": "COMPLEX",
    "What is quantum computing?": "COMPLEX",
    "What are the legal implications of breach of contract?": "COMPLEX",
    "Describe the process of cellular respiration.": "SIMPLE",
    "What are the different types of machine learning algorithms?": "COMPLEX",
    "Explain the significance of the Magna Carta in legal history.": "COMPLEX",
    "How does blockchain technology function?": "COMPLEX",
    "What are the key principles of constitutional law?": "COMPLEX",
    "Describe the water cycle in nature.": "SIMPLE",
    "What are the main functions of the human brain?": "COMPLEX",
    "Explain the concept of supply and demand in economics.": "SIMPLE",
    "What are the different types of renewable energy sources?": "SIMPLE",
    "How does the immune system protect the body from pathogens?": "COMPLEX",
    "What are the legal rights of tenants in rental agreements?": "COMPLEX",
    "Describe the process of DNA replication.": "COMPLEX",
    "What are the main components of a computer system?": "SIMPLE",
    "Explain the principles of criminal law.": "COMPLEX",
    "What are the effects of global warming on ecosystems?": "COMPLEX",
    "How does the nervous system transmit signals?": "COMPLEX",
    "What are the different types of financial markets?": "SIMPLE",
    "Explain the concept of intellectual property law.": "COMPLEX",
    "Describe the process of mitosis in cell division.": "COMPLEX",
    "Explain the bulking stress and what's its inpact on the heavy materials.":"COMPLEX",
    "What is Newton's first law of motion?": "SIMPLE",
    "How does a combustion engine work?": "COMPLEX",
    "What is the difference between speed and velocity?": "SIMPLE",
    "Explain the working principle of a nuclear reactor.": "COMPLEX",
    "What are the basic units of measurement in SI system?": "SIMPLE",
    "How does machine learning differ from traditional programming?": "COMPLEX",
    "What is photosynthesis used for in plants?": "SIMPLE",
    "Explain stress-strain behavior of ductile materials.": "COMPLEX",
    "What is the purpose of an operating system?": "SIMPLE",
    "How does finite element analysis help in structural design?": "COMPLEX",
    "What are the properties of an ideal gas?": "SIMPLE",
    "Explain the difference between AC and DC power transmission.": "SIMPLE",
    "How does corrosion affect marine structures?": "COMPLEX",
    "What is the function of red blood cells?": "SIMPLE",
    "Explain the concept of entropy in thermodynamics.": "COMPLEX",
    "What are renewable and non-renewable resources?": "SIMPLE",
    "How do tides affect ship operations?": "COMPLEX",
    "What is the role of classification societies in maritime safety?": "COMPLEX",
    "Explain fatigue failure and its significance in engineering design.": "COMPLEX"
}



In [2]:

records = []
LLM_MODEL=os.getenv("LLM_MODEL1","qwen2.5:1.5b")
for q in QUERIES:
    result = route_query(q)
    # print("query is : ", q)
    # print("result is : ", result)

    records.append({
        "model": LLM_MODEL,
        "query": q,
        "predicted_complexity": result.get("complexity"),
        "confidence": result.get("confidence"),
        "domain": result.get("domain"),
        "reason": result.get("reason"),
        "ground_truth_complexity": QUERIES[q]
    })

df1 = pd.DataFrame(records)


In [3]:
df1.head(10)

,model,query,predicted_complexity,confidence,domain,reason,ground_truth_complexity
0,qwen2.5:1.5b,What is the capital of France?,SIMPLE,1.00,GENERAL,The user is asking for basic factual knowledge...,SIMPLE
1,qwen2.5:1.5b,Explain ship classification and statutory regu...,COMPLEX,0.95,TECHNICAL,The user is asking for an explanation of ship ...,COMPLEX
2,qwen2.5:1.5b,How does compound interest work?,SIMPLE,0.90,FINANCE,The user is asking about a basic financial con...,SIMPLE
3,qwen2.5:1.5b,What are the symptoms of diabetes?,SIMPLE,0.90,MEDICAL,"The user is asking about a medical condition, ...",SIMPLE
4,qwen2.5:1.5b,Solve the equation x^2 - 4x + 4 = 0,SIMPLE,1.00,MATHEMATICS,The user is asking for a simple solution to an...,SIMPLE
5,qwen2.5:1.5b,Describe the process of photosynthesis.,COMPLEX,0.95,TECHNICAL,The user is asking for a detailed explanation ...,SIMPLE
6,qwen2.5:1.5b,What are the main causes of climate change?,COMPLEX,0.95,GENERAL,The user is asking for a broad overview of the...,COMPLEX
7,qwen2.5:1.5b,Explain the theory of relativity.,COMPLEX,0.95,TECHNICAL,The user is asking for a detailed explanation ...,COMPLEX
8,qwen2.5:1.5b,How do vaccines work?,SIMPLE,0.90,OTHER,The user is asking about a basic factual quest...,COMPLEX
9,qwen2.5:1.5b,What is quantum computing?,SIMPLE,0.90,TECHNICAL,The user is asking about a basic concept in co...,COMPLEX


In [5]:
df1.to_csv("qwen2.5_1.5.csv",index=False)

In [4]:
df1["is_correct"] = (
    df1["predicted_complexity"] == df1["ground_truth_complexity"]
)
# print(df1["is_correct"])
accuracy = df1["is_correct"].mean()
print("Accuracy:", accuracy)

Accuracy: 0.66


In [10]:

records = []
LLM_MODEL=os.getenv("LLM_MODEL1","qwen2.5:1.5b-instruct")
for q in QUERIES:
    result = route_query(q)
    # print("query is : ", q)
    # print("result is : ", result)

    records.append({
        "model": LLM_MODEL,
        "query": q,
        "predicted_complexity": result.get("complexity"),
        "confidence": result.get("confidence"),
        "domain": result.get("domain"),
        "reason": result.get("reason"),
        "ground_truth_complexity": QUERIES[q]
    })

df2 = pd.DataFrame(records)


In [11]:
df2["is_correct"] = (
    df2["predicted_complexity"] == df2["ground_truth_complexity"]
)
# print(df1["is_correct"])
accuracy = df2["is_correct"].mean()
print("Accuracy:", accuracy)

Accuracy: 0.66


In [12]:
df2.to_csv("qwen2.5_1.5-instruct.csv",index=False)

In [6]:

records = []
LLM_MODEL=os.getenv("LLM_MODEL1","gemma3:1b")
for q in QUERIES:
    result = route_query(q)
    # print("query is : ", q)
    # print("result is : ", result)

    records.append({
        "model": LLM_MODEL,
        "query": q,
        "predicted_complexity": result.get("complexity"),
        "confidence": result.get("confidence"),
        "domain": result.get("domain"),
        "reason": result.get("reason"),
        "ground_truth_complexity": QUERIES[q]
    })

df3 = pd.DataFrame(records)


In [7]:
df3["is_correct"] = (
    df3["predicted_complexity"] == df3["ground_truth_complexity"]
)
# print(df1["is_correct"])
accuracy = df3["is_correct"].mean()
print("Accuracy:", accuracy)

Accuracy: 0.7


In [9]:
df3.to_csv("gemma3_1b.csv",index=False)

In [2]:
REASONING_KEYWORDS = [
    "why", "how", "analyze", "analysis", "impact", "effect",
    "evaluate", "assess", "critically", "reason","explain"
]

PLANNING_KEYWORDS = [
    "how to", "steps", "roadmap", "plan", "strategy", "approach",
    "design", "build", "implement"
]

COMPARISON_KEYWORDS = [
    "compare", "difference", "vs", "versus", "pros and cons",
    "advantages", "disadvantages"
]

STRUCTURED_OUTPUT_KEYWORDS = [
    "table", "list", "steps", "bullet", "framework", "flow",
    "diagram", "structure"
]

AMBIGUOUS_REFERENCES = [
    "this", "that", "these", "those", "above", "earlier", "previous",
    "mentioned", "it"
]

CONJUNCTIONS = [
    " and ", " or ", " also ", " as well as ", " along with "
]

In [3]:
import json
import re
from typing import Literal, Optional
from pydantic import BaseModel

from langchain_ollama import ChatOllama
import os
import requests

MODEL_API = os.getenv("MODEL_API", "http://localhost:11434")
# EMBED_MODEL = os.getenv("EMBED_MODEL", "qwen3-embedding:0.6b")
# LLM_MODEL=os.getenv("LLM_MODEL","qwen2.5:3b-instruct-q8_0")
# LLM_MODEL=os.getenv("LLM_MODEL1","gemma3:1b")
# LLM_MODEL=os.getenv("LLM_MODEL1","qwen2.5:1.5b-instruct")
LLM_MODEL=os.getenv("LLM_MODEL1","qwen2.5:1.5b")
# LLM_MODEL=os.getenv("LLM_MODEL1","qwen2.5:0.5b")   
# OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://ollama:11434") 


QueryComplexity = Literal["SIMPLE", "COMPLEX"]
domainType = Literal["LAW", "TECHNICAL", "FINANCE", "MEDICAL","MATHEMATICS", "GENERAL", "OTHER"]


class RouterOutput(BaseModel):
    complexity: QueryComplexity
    confidence: float
    reason: str
    domain: domainType


def safe_json_parse(text: str):
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if match:
            return json.loads(match.group())
        raise ValueError("Invalid JSON from LLM")


def route_query(query: str) -> RouterOutput:
    prompt = f"""
You are a routing classifier for a RAG system.

Your task is to classify a user query by:
1) Complexity
2) Domain (knowledge area)

RULES:
- If the reason implies basic factual knowledge → complexity MUST be SIMPLE
- If the reason implies analysis or deep reasoning → complexity MUST be COMPLEX
- Domain MUST match the reason
- NEVER contradict your own reason

Confidence score:
- this is a number between 0 and 1 decimal number representing how confident you are in your classifcation.
- any complexity you classify always tell me how confident on that classification.

Domain:
- LAW
- TECHNICAL
- FINANCE
- MEDICAL
- MATHEMATICS
- GENERAL
- OTHER

Return ONLY valid JSON in this format:
{{
  "complexity": "SIMPLE" | "COMPLEX",
  "confidence": number,
  "domain": "LAW" | "TECHNICAL" | "FINANCE" | "MEDICAL" | "MATHEMATICS" | "GENERAL" | "OTHER",
  "reason": "brief justification"
}}
anylize the reason you give and find complexity and confidence based on the query justification.

User query:
"{query}"
"""


    try:
        data = {
            "model": LLM_MODEL,
            "prompt": prompt,
            "format": "json",
            "options": {
                "temperature": 0,
                "top_p": 0.9,
                "repeat_penalty": 1.1,
                "num_predict": 256
            }
        }

        response = requests.post(f"{MODEL_API}/api/generate", json=data, timeout=30)

        # --- Collect streamed chunks ---
        fragments = []
        for line in response.text.strip().splitlines():
            obj = json.loads(line)
            if "response" in obj:
                fragments.append(obj["response"])

        # --- Join all fragments into one JSON string ---
        full_text = "".join(fragments)

        # --- Parse the complete JSON ---
        parsed = json.loads(full_text)

        # return RouterOutput(**parsed)
        return parsed
    except Exception as e:
        print("[Router Error]", e)
        return RouterOutput(
            complexity="SIMPLE",
            confidence=0.5,
            reason="Fallback due to routing error"
        )

In [4]:
SIMPLE_FACT_PATTERNS = [
    r"what is",
    r"who is",
    r"when did",
    r"where is",
    r"name the",
    r"define ",
    r"formula for",
    r"symbol for",
    r"capital of",
]

In [5]:
def heuristic_complexity_score(query: str):
    q = query.lower()
    words = q.split()

    score = 0
    reasons = []

    #  Length 
    if len(words) > 14:
        score += 1
        reasons.append("Long query")

    if len(query) > 90:
        score += 1
        reasons.append("High character length")

    # Reasoning calculating
    reasoning_hits = sum(1 for k in REASONING_KEYWORDS if k in q)
    if reasoning_hits >= 2:
        score += 3
        reasons.append("Multiple reasoning intents")
    elif reasoning_hits == 1:
        score += 1.5
        reasons.append("Single reasoning intent")
    def keyword_hit(keywords):
        return any(k in q for k in keywords)
 
    if keyword_hit(PLANNING_KEYWORDS):
        score += 2
        reasons.append("Planning task")

    if keyword_hit(COMPARISON_KEYWORDS):
        score += 2
        reasons.append("Comparison task")

    # Conjunction finding
    if any(c in q for c in [" and ", " or "]):
        if reasoning_hits == 0:
            score += 0.5
            reasons.append("Multiple factual parts")
        else:
            score += 2
            reasons.append("Multi-part reasoning")

    # --- Math / symbols ---
    if re.search(r"[=<>+\-*/%]", query):
        score += 1
        reasons.append("Math or logical symbols")

    # --- Negative scoring ---
    if len(words) < 8 and reasoning_hits == 0:
        score -= 1
        reasons.append("Short factual query")

    if any(p in q for p in SIMPLE_FACT_PATTERNS) and reasoning_hits == 0:
        score -= 2
        reasons.append("Pure factual intent")

    # --- Confidence ---
    confidence_rule = min(0.95, 0.45 + score * 0.08)

    # --- Routing ---
    if score >= 6:
        return {"complexity": "COMPLEX", "confidence": confidence_rule, "score": score, "reasons": reasons}

    if score <= 2:
        return {"complexity": "SIMPLE", "confidence": confidence_rule, "score": score, "reasons": reasons}

    # --- Borderline → LLM ---
    llm_resp = route_query(query=query)
    confidence_llm = llm_resp.get("confidence", 0.5)

    w_avg = 0.55 * confidence_rule + 0.45 * confidence_llm
    complexity = "COMPLEX" if w_avg >= 0.65 else "SIMPLE"

    return {
        "complexity": complexity,
        "confidence": round(w_avg, 2),
        "score": score,
        "reasons": reasons
    }
# response = heuristic_complexity_score("Explain the process of machine learning and its real-world applications.")
# response = heuristic_complexity_score("Analyze the causes and effects of climate change.")
# response = heuristic_complexity_score("Name  largest planet in solar system  whats its name.")
# response = heuristic_complexity_score("Compare renewable and non-renewable energy sources in terms of sustainability and cost.")
# response = heuristic_complexity_score("Why is cybersecurity risk management critical for financial institutions?")
# response = heuristic_complexity_score("Analyze the role of ship classification societies in maritime safety and regulation.")
response = heuristic_complexity_score("Explain how APIs enable communication between distributed software systems.")

response

{'complexity': 'COMPLEX',
 'confidence': 0.81,
 'score': 3,
 'reasons': ['Multiple reasoning intents']}

In [ ]:
GROUND_TRUTH_QUERIES = [
  {"query": "Name the largest planet in the solar system.", "complexity": "SIMPLE"},
  {"query": "What is the capital of France?", "complexity": "SIMPLE"},
  {"query": "Who wrote Romeo and Juliet?", "complexity": "SIMPLE"},
  {"query": "What is 15 multiplied by 4?", "complexity": "SIMPLE"},
  {"query": "What gas do plants absorb from the atmosphere?", "complexity": "SIMPLE"},
  {"query": "How many continents are there on Earth?", "complexity": "SIMPLE"},
  {"query": "What is the boiling point of water in Celsius?", "complexity": "SIMPLE"},
  {"query": "Who was the first President of the United States?", "complexity": "SIMPLE"},
  {"query": "What is the chemical symbol for gold?", "complexity": "SIMPLE"},
  {"query": "Which ocean is the largest?", "complexity": "SIMPLE"},
  {"query": "What is the square root of 64?", "complexity": "SIMPLE"},
  {"query": "What color do you get by mixing red and white?", "complexity": "SIMPLE"},
  {"query": "How many days are there in a leap year?", "complexity": "SIMPLE"},
  {"query": "What planet is known as the Red Planet?", "complexity": "SIMPLE"},
  {"query": "What is the freezing point of water in Celsius?", "complexity": "SIMPLE"},
  {"query": "Who painted the Mona Lisa?", "complexity": "SIMPLE"},
  {"query": "What is the tallest land animal?", "complexity": "SIMPLE"},
  {"query": "Which language is primarily spoken in Brazil?", "complexity": "SIMPLE"},
  {"query": "What is the currency of Japan?", "complexity": "SIMPLE"},
  {"query": "How many sides does a triangle have?", "complexity": "SIMPLE"},
  {"query": "What is the process by which plants make food?", "complexity": "SIMPLE"},
  {"query": "What is the largest mammal on Earth?", "complexity": "SIMPLE"},
  {"query": "Who discovered gravity?", "complexity": "SIMPLE"},
  {"query": "What is the capital of Canada?", "complexity": "SIMPLE"},
  {"query": "What instrument is used to measure temperature?", "complexity": "SIMPLE"},
  {"query": "Explain how photosynthesis works in plants and analyze why it is essential for sustaining life on Earth.", "complexity": "COMPLEX"},
  {"query": "Analyze the main causes of climate change and explain their effects on global ecosystems and biodiversity.", "complexity": "COMPLEX"},
  {"query": "Compare and contrast democracy and authoritarian forms of government by analyzing their structures and outcomes.", "complexity": "COMPLEX"},
  {"query": "Explain how blockchain technology works and evaluate its real-world applications across different industries.", "complexity": "COMPLEX"},
  {"query": "Evaluate the impact of artificial intelligence on future job markets and workforce skill requirements.", "complexity": "COMPLEX"},
  {"query": "Explain the theory of relativity and analyze its significance in modern physics and scientific research.", "complexity": "COMPLEX"},
  {"query": "Discuss the ethical implications of genetic engineering in humans and evaluate potential societal risks.", "complexity": "COMPLEX"},
  {"query": "Analyze the causes of World War I and explain their long-term political and economic consequences.", "complexity": "COMPLEX"},
  {"query": "Explain how machine learning models are trained and evaluated and why data quality affects their performance.", "complexity": "COMPLEX"},
  {"query": "Describe the process of natural selection and analyze real-world examples that demonstrate evolutionary change.", "complexity": "COMPLEX"},
  {"query": "Compare renewable and non-renewable energy sources in terms of sustainability, cost, and environmental impact.", "complexity": "COMPLEX"},
  {"query": "Explain the role of the United Nations in maintaining global peace and resolving international conflicts.", "complexity": "COMPLEX"},
  {"query": "Discuss the economic causes of inflation and analyze how governments attempt to control it.", "complexity": "COMPLEX"},
  {"query": "Analyze the advantages and disadvantages of remote work and its impact on productivity and work-life balance.", "complexity": "COMPLEX"},
  {"query": "Explain how the human immune system responds to viral infections and why immune memory is important.", "complexity": "COMPLEX"},
  {"query": "Describe the architecture of a modern operating system and explain how its components interact.", "complexity": "COMPLEX"},
  {"query": "Evaluate the social impact of social media on mental health and analyze contributing psychological factors.", "complexity": "COMPLEX"},
  {"query": "Explain the water cycle and analyze its importance to Earth’s climate and ecosystems.", "complexity": "COMPLEX"},
  {"query": "Analyze the differences between SQL and NoSQL databases and explain when each approach is preferable.", "complexity": "COMPLEX"},
  {"query": "Discuss the challenges of space exploration and colonization and analyze technological and ethical risks.", "complexity": "COMPLEX"},
  {"query": "Explain how economic supply and demand interact and analyze how they affect market prices.", "complexity": "COMPLEX"},
  {"query": "Analyze the role of education in economic development and its long-term impact on societal growth.", "complexity": "COMPLEX"},
  {"query": "Describe how neural networks mimic the human brain and explain how they learn complex patterns.", "complexity": "COMPLEX"},
  {"query": "Explain the causes of cybersecurity attacks and analyze strategies for their prevention.", "complexity": "COMPLEX"},
  {"query": "Discuss how globalization affects developing countries and analyze both economic benefits and risks.", "complexity": "COMPLEX"},
  {"query": "Explain the life cycle of a star from formation to supernova and analyze key physical processes.", "complexity": "COMPLEX"},
  {"query": "Analyze the impact of pandemics on global economies and explain long-term recovery challenges.", "complexity": "COMPLEX"},
  {"query": "Describe the principles of object-oriented programming and explain how they improve software design.", "complexity": "COMPLEX"},
  {"query": "Explain how encryption ensures data security and analyze its role in modern digital systems.", "complexity": "COMPLEX"},
  {"query": "Discuss psychological theories of motivation and analyze how they influence human behavior.", "complexity": "COMPLEX"},
  {"query": "Analyze the relationship between culture and language and explain how they influence each other.", "complexity": "COMPLEX"},
  {"query": "Explain how renewable energy technologies reduce carbon emissions and support environmental sustainability.", "complexity": "COMPLEX"},
  {"query": "Evaluate the pros and cons of nuclear energy and analyze its role in future energy strategies.", "complexity": "COMPLEX"},
  {"query": "Describe the software development life cycle and explain how each phase contributes to quality software.", "complexity": "COMPLEX"},
  {"query": "Explain the effects of deforestation on biodiversity and analyze long-term environmental consequences.", "complexity": "COMPLEX"},
  {"query": "Analyze how interest rates influence economic growth and affect consumer and business behavior.", "complexity": "COMPLEX"},
  {"query": "Discuss the role of ethics in artificial intelligence development and analyze potential risks of misuse.", "complexity": "COMPLEX"},
  {"query": "Explain how memory management works in operating systems and why it is critical for performance.", "complexity": "COMPLEX"},
  {"query": "Analyze the causes of income inequality across nations and explain contributing economic factors.", "complexity": "COMPLEX"},
  {"query": "Describe how cloud computing supports scalable applications and analyze its business advantages.", "complexity": "COMPLEX"},
  {"query": "Explain the impact of urbanization on the environment and analyze challenges for sustainable cities.", "complexity": "COMPLEX"},
  {"query": "Discuss how big data analytics improves business decision-making and analyze implementation challenges.", "complexity": "COMPLEX"},
  {"query": "What is the capital of Italy?", "complexity": "SIMPLE"},
  {"query": "Who invented the telephone?", "complexity": "SIMPLE"},
  {"query": "What is 9 plus 11?", "complexity": "SIMPLE"},
  {"query": "Name the largest ocean on Earth.", "complexity": "SIMPLE"},
  {"query": "What color is chlorophyll?", "complexity": "SIMPLE"},
  {"query": "How many planets are in the solar system?", "complexity": "SIMPLE"},
  {"query": "What is the currency of the UK and US?", "complexity": "SIMPLE"},
  {"query": "Who wrote Harry Potter and why?", "complexity": "SIMPLE"},
  {"query": "What is the freezing point of water?", "complexity": "SIMPLE"},
  {"query": "Which animal is known as the king of the jungle?", "complexity": "SIMPLE"},
  {"query": "What does CPU stand for?", "complexity": "SIMPLE"},
  {"query": "What is the square of 12 and 13?", "complexity": "SIMPLE"},
  {"query": "What gas do humans breathe in to survive?", "complexity": "SIMPLE"},
  {"query": "Which country is famous for sushi?", "complexity": "SIMPLE"},
  {"query": "How many hours and minutes are there in a day?", "complexity": "SIMPLE"},
  {"query": "What planet is closest to the sun?", "complexity": "SIMPLE"},
  {"query": "What is the primary color mixed with blue to make green?", "complexity": "SIMPLE"},
  {"query": "What is the chemical symbol for oxygen and sulphur?", "complexity": "SIMPLE"},
  {"query": "Who was the first man on the moon and sun?", "complexity": "SIMPLE"},
  {"query": "What device is used to measure weight?", "complexity": "SIMPLE"},
  {"query": "Which programming language is mainly used for web styling?", "complexity": "SIMPLE"},
  {"query": "What is the boiling point of water and ice?", "complexity": "SIMPLE"},
  {"query": "What shape has four equal sides?", "complexity": "SIMPLE"},
  {"query": "Which organ pumps blood in the human body?", "complexity": "SIMPLE"},
  {"query": "What does URL and LRU stand for?", "complexity": "SIMPLE"},
  {"query": "Explain how machine learning models are trained, validated, and evaluated using real-world datasets.", "complexity": "COMPLEX"},
  {"query": "Analyze the causes and long-term economic effects of inflation on developing countries.", "complexity": "COMPLEX"},
  {"query": "Compare renewable and non-renewable energy sources in terms of sustainability, cost, and environmental impact.", "complexity": "COMPLEX"},
  {"query": "Explain why cybersecurity risk management is critical for financial institutions and how it reduces losses.", "complexity": "COMPLEX"},
  {"query": "Analyze how climate change affects agriculture, water resources, and global food security.", "complexity": "COMPLEX"},
  {"query": "Explain the differences between supervised, unsupervised, and reinforcement learning with examples.", "complexity": "COMPLEX"},
  {"query": "Compare SQL and NoSQL databases and explain when each should be used.", "complexity": "COMPLEX"},
  {"query": "Analyze the ethical implications of artificial intelligence in healthcare decision-making.", "complexity": "COMPLEX"},
  {"query": "Explain how operating systems manage memory and processes simultaneously.", "complexity": "COMPLEX"},
  {"query": "Discuss the advantages and disadvantages of cloud computing for enterprise applications.", "complexity": "COMPLEX"},
  {"query": "Analyze how social media algorithms influence user behavior and information consumption.", "complexity": "COMPLEX"},
  {"query": "Explain the role of DNA replication and mutation in the process of evolution.", "complexity": "COMPLEX"},
  {"query": "Compare capitalism and socialism by analyzing their economic structures and social outcomes.", "complexity": "COMPLEX"},
  {"query": "Explain how blockchain technology ensures transparency and security in financial transactions.", "complexity": "COMPLEX"},
  {"query": "Analyze the impact of globalization on employment patterns in emerging economies.", "complexity": "COMPLEX"},
  {"query": "Explain why supply and demand fluctuations can lead to market instability.", "complexity": "COMPLEX"},
  {"query": "Compare different software development life cycle models and explain their use cases.", "complexity": "COMPLEX"},
  {"query": "Analyze how interest rate changes affect inflation, investment, and consumer spending.", "complexity": "COMPLEX"},
  {"query": "Explain how neural networks learn complex patterns using backpropagation and gradient descent.", "complexity": "COMPLEX"},
  {"query": "Discuss the challenges and risks involved in space exploration and long-term colonization.", "complexity": "COMPLEX"},
  {"query": "Analyze the causes and consequences of data breaches in large organizations.", "complexity": "COMPLEX"},
  {"query": "Explain how natural selection and genetic variation drive species adaptation.", "complexity": "COMPLEX"},
  {"query": "Compare nuclear energy with renewable energy sources in terms of safety and scalability.", "complexity": "COMPLEX"},
  {"query": "Explain how recommendation systems work and why they improve user engagement.", "complexity": "COMPLEX"},
  {"query": "Analyze the role of education systems in reducing income inequality.", "complexity": "COMPLEX"},
  {"query": "Explain how sound waves travel through air and why pitch and volume can change.", "complexity": "COMPLEX"},
  {"query": "Discuss how urbanization affects ecosystems, climate, and human health.", "complexity": "COMPLEX"},
  {"query": "Analyze the pros and cons of remote work for productivity and employee well-being.", "complexity": "COMPLEX"},
  {"query": "Explain how encryption algorithms protect data confidentiality and integrity.", "complexity": "COMPLEX"},
  {"query": "Compare different types of cybersecurity attacks and strategies to mitigate them.", "complexity": "COMPLEX"},
  {"query": "Analyze the impact of pandemics on global supply chains and economic stability.", "complexity": "COMPLEX"},
  {"query": "Explain how APIs enable communication between distributed software systems.", "complexity": "COMPLEX"},
  {"query": "Discuss the environmental and economic impacts of deforestation.", "complexity": "COMPLEX"},
  {"query": "Analyze why bias can emerge in machine learning models and how it can be reduced.", "complexity": "COMPLEX"},
  {"query": "Explain the planning considerations involved in migrating legacy systems to the cloud.", "complexity": "COMPLEX"},
  {"query": "Compare centralized and decentralized systems in terms of reliability and security.", "complexity": "COMPLEX"},
  {"query": "Analyze the role of ship classification societies in maritime safety and regulation.", "complexity": "COMPLEX"},
  {"query": "Explain how big data analytics supports strategic business decision-making.", "complexity": "COMPLEX"},
  {"query": "Discuss why ethical governance is necessary for large-scale AI deployment.", "complexity": "COMPLEX"},
  {"query": "Analyze the factors influencing consumer behavior in digital marketplaces.", "complexity": "COMPLEX"},
  {"query": "Explain how memory leaks occur in software systems and how they can be prevented.", "complexity": "COMPLEX"},
  {"query": "Compare short-term and long-term economic impacts of government stimulus policies.", "complexity": "COMPLEX"},
  {"query": "Explain how climate models are used to predict future environmental changes.", "complexity": "COMPLEX"},
  {"query": "Analyze the challenges of maintaining data privacy in cloud-based applications.", "complexity": "COMPLEX"},
  {"query": "Discuss how automation and AI are transforming traditional manufacturing industries.", "complexity": "COMPLEX"},
  {"query": "Explain the relationship between risk management and organizational resilience.", "complexity": "COMPLEX"},
  {"query": "Analyze how technological innovation drives competitive advantage in businesses.", "complexity": "COMPLEX"},
  {"query": "Compare different approaches to software testing and their effectiveness.", "complexity": "COMPLEX"}
]
records = []
import pandas as pd
for item in GROUND_TRUTH_QUERIES:
    query = item["query"]
    complexity = item["complexity"]
    records.append({
        "query":query,
        "complexity":complexity
    })
    
df1 = pd.DataFrame(records)
# df1[25:49]

In [7]:
records = [] 
import pandas as pd
for item in GROUND_TRUTH_QUERIES:
    query = item["query"]
    complexity = item["complexity"]
    
    result = heuristic_complexity_score(query)
    records.append({
        "query":query,
        "GROUND_TRUTH":complexity,
        "predicted_complexity": result["complexity"],
        "confidence":result["confidence"],
        "score":result["score"],
        "reason":result["reasons"]
    })
df4 = pd.DataFrame(records)


In [8]:
df4.tail(20)

,query,GROUND_TRUTH,predicted_complexity,confidence,score,reason
120,Explain how encryption algorithms protect data...,COMPLEX,COMPLEX,0.90,5.0,"[Multiple reasoning intents, Multi-part reason..."
121,Compare different types of cybersecurity attac...,COMPLEX,COMPLEX,0.76,2.5,"[Comparison task, Multiple factual parts]"
122,Analyze the impact of pandemics on global supp...,COMPLEX,COMPLEX,0.90,5.0,"[Multiple reasoning intents, Multi-part reason..."
123,Explain how APIs enable communication between ...,COMPLEX,COMPLEX,0.81,3.0,[Multiple reasoning intents]
124,Discuss the environmental and economic impacts...,COMPLEX,COMPLEX,0.81,3.5,"[Single reasoning intent, Multi-part reasoning]"
125,Analyze why bias can emerge in machine learnin...,COMPLEX,COMPLEX,0.93,6.0,"[Long query, Multiple reasoning intents, Multi..."
126,Explain the planning considerations involved i...,COMPLEX,COMPLEX,0.83,3.5,"[Single reasoning intent, Planning task]"
127,Compare centralized and decentralized systems ...,COMPLEX,COMPLEX,0.76,2.5,"[Comparison task, Multiple factual parts]"
128,Analyze the role of ship classification societ...,COMPLEX,COMPLEX,0.83,3.5,"[Single reasoning intent, Multi-part reasoning]"
129,Explain how big data analytics supports strate...,COMPLEX,COMPLEX,0.85,4.0,"[Multiple reasoning intents, Math or logical s..."


In [ ]:
df4["is_correct"] = (
    df4["predicted_complexity"] == df4["GROUND_TRUTH"]
)
accuracy = df4["is_correct"].mean()

                                
print("Accuracy:", accuracy)    

Accuracy: 0.9714285714285714


In [24]:
df4.to_csv("rule_based_routing.csv",index=False)